## Extraer datos mediante la API de inmobiliaria

Pagina donde se hizo el webscraping: https://www.argenprop.com/ 

Paises admitidos: Argentina, Chile, Uruguay, Brasil(1propiedad)

In [3]:
#Librerias
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime
import pytz
from unidecode import unidecode
import re

## 1. Creamos la conexión con la API para extraer los datos

In [4]:
%%time

#url = 'http://reffindr-alb-1167121448.us-east-1.elb.amazonaws.com:4155/argenprop'
url = 'https://i004-reffindr-back-python-dev.onrender.com/argenprop'

# Parámetros de la solicitud
params = {'pais': 'argentina', 'limite': 300}
response = requests.get(url, params=params)
print(response.status_code)

200
CPU times: total: 46.9 ms
Wall time: 5min 16s


In [7]:
data = response.json()

In [8]:
df = pd.DataFrame(data)
df

,Bathrooms,Bedrooms,CountryName,Description,Environments,Latitude,Longitude,Price,Seniority,StateName,Title,img
0,2,3,Argentina,"Casa en Barrio Privado Camino Real, cuenta con...",4,"-34,48684","-58,58234",USD 2.300,27,Buenos Aires,Av. Camino Real Morón San Fernando 1500,[https://static1.sosiva451.com/08910661/ea4bac...
1,2,3,Argentina,Alquilo amplio Duplex en Florencio varela comb...,4,"-34,79154","-58,27525",$ 490.000,10,Buenos Aires,Combate De San Lorenzo 1700,[https://static1.sosiva451.com/28339561/8cca67...
2,2,4,Argentina,DUEÑO ALQUILA\r\n\n •SIN EXPENSAS\r\n •SIN GAS...,5,0.0,0.0,USD 1.100,15,Buenos Aires,casa en lujan con pileta alquiler anual,[https://static1.sosiva451.com/09149461/6abd53...
3,2,4,Argentina,Imponente casa de estilo en la esquina de Alve...,5,"-32,94819","-60,66076",$ 2.300.000,90,Santa Fe,Mendoza 2300,[https://static1.sosiva451.com/22799461/e73d3f...
4,5,5,Argentina,Exclusiva Casa en Alquiler en Puertos del Lago...,7,"-34,318497","-58,742558",USD 7.500,0,Buenos Aires,Puertos del Lago Barrio Marinas Escobar Alquiler,[https://static1.sosiva451.com/48236461/e64ab1...
...,...,...,...,...,...,...,...,...,...,...,...,...
295,4,5,Argentina,Dueño directo \n Deportiva I- L146\n\n Barrio...,6,"-37,0089","-56,80412",USD 6.500,1,Buenos Aires,Costa Esmeralda-deportivo I,[https://static1.sosiva451.com/62641321/178b35...
296,5,None,Argentina,EDIFICIO MONTEVIDEO 359\r\n Nueva Cordoba\r\n\...,None,"-31,420574","-64,191246",$ 2.000.000,0,Cordoba,Montevideo 300,[https://static1.sosiva451.com/98857951/d3b8d0...
297,3,3,Argentina,Alquiler. \n Semipiso de 4 ambientes en edific...,4,"-34,48799","-58,49957",USD 1.600,0,Buenos Aires,Vicente Fidel López 100,[https://static1.sosiva451.com/26151061/71045e...
298,5,3,Argentina,Hermosa y amplia Casa desarrollada en dos plan...,4,"-34,36595","-58,7351",USD 1.350,6,Buenos Aires,La Pista 100,[https://static1.sosiva451.com/26798261/62d8a1...


## 1.1 Creación de Tabla de States

In [9]:
data_states = {
    1: "Buenos Aires",
    2: "Catamarca",
    3: "Chaco",
    4: "Chubut",
    5: "Córdoba",
    6: "Corrientes",
    7: "Entre Ríos",
    8: "Formosa",
    9: "Jujuy",
    10: "La Pampa",
    11: "La Rioja",
    12: "Mendoza",
    13: "Misiones",
    14: "Neuquén",
    15: "Río Negro",
    16: "Salta",
    17: "San Juan",
    18: "San Luis",
    19: "Santa Cruz",
    20: "Santa Fe",
    21: "Santiago del Estero",
    22: "Tierra del Fuego",
    23: "Tucumán",
    24: "Ciudad Autónoma de Buenos Aires",
}

# Crear un DataFrame con los estados
States = pd.DataFrame(data_states.items(), columns=["Id", "StateName"])

In [10]:
States

,Id,StateName
0,1,Buenos Aires
1,2,Catamarca
2,3,Chaco
3,4,Chubut
4,5,Córdoba
5,6,Corrientes
6,7,Entre Ríos
7,8,Formosa
8,9,Jujuy
9,10,La Pampa
